## This is the initial network design for chromstem-net

In [79]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

This is the dataset generator class, to be loaded into the DataLoader tool

This is adapted from the pytorch tutorial: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [81]:
class ChromstemDataset(Dataset):
    """Chromstem Dataset"""
    
    def __init__(self,csv_file,root_dir,transform=None):
        """
        Args:
            csv_file (string): Master CSV file that will hold all the data
            root_dir (string): Directory where the datasets are labeled
            transform (callable, optional): Optional transformations to samples
        """
        
        f = open(csv_file,'r')
        csv_length = len(f.readlines()[-1].split(','))
        f.close()
        col_names = ['File'] + ['Nucl%d' % i for i in range(csv_length-1)]
        self.nucl_coords_frame_ = pd.read_csv(csv_file,engine='python',header=None,names=col_names)
        self.root_dir_ = root_dir
        self.transform_ = transform
    
    def read_chromstem(self,fnme):
        x,y,z,rho = np.loadtxt(fnme,comments='#',unpack=True)
        tensor = torch.zeros_like(torch.empty(1,int(x[0]),int(y[0]),int(z[0])))
    
        for i,arr in enumerate(zip(x[1:],y[1:],z[1:])):
            tensor[0,int(arr[0]),int(arr[1]),int(arr[2])] = rho[i]
        
        # Convert to sparse matrix as data will be sparse
        #tensor = torch.Sparse.FloatTensor(tensor)
        return tensor
    
    def __len__(self):
        return(len(self.nucl_coords_frame_))
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        chromstem_name = os.path.join(self.root_dir_,self.nucl_coords_frame_.iloc[idx,0])
        chromstem = self.read_chromstem(chromstem_name)
        nucl_coords = self.nucl_coords_frame_.iloc[idx,1:]
        nucl_coords = np.asarray([nucl_coords])
        nucl_coords = nucl_coords.astype('float').reshape(-1,3)
        num_nucls = int(chromstem_name.split('/data/')[1].split('/')[0])
        
        sample = {'chromstem' : chromstem, 'nucl_coords' : nucl_coords, 'num_nucls' : num_nucls}
        
        if self.transform_:
            sample = self.transform_(sample)
            
        return sample
        

Here, would be useful to provide a plotting function for samples in the dataset.
The main input to this function is any sample from the dataset and it would output the density and "centromere" in a 3D plot.
To do so accurately, I am going to use the ipyvolume widget.

In [82]:
import numpy as np
import ipyvolume as ipv

In [83]:
def plot_sample(sample):
    ipv.figure()
    # Plot voxels of density
    ipv.scatter(sample['chromstem'][:,0],sample['chromstem'][:,1],sample['chromstem'][:,2], 
                color='blue',size=10,marker='box',opacity=0.5)
    
    # Plot nucleosome centers
    ipv.scatter(sample['nucl_coords'][:,0],sample['nucl_coords'][:,1],sample['nucl_coords'][:,2],
               color='red',size=5,marker='sphere')
    
    ipv.show()
    return

This is a quick test of the widget and plotting function before the data is loaded

In [84]:
sample = {'chromstem' :  np.asarray([[1,2,3]]), 'nucl_coords' : np.asarray([[0,1,2],[5,10,0.2],[10,3.3,2]])}
plot_sample(sample)

Here is where the dataset is loaded in. The .csv file should be located in the current directory, so the root is '.'

In [85]:
chromstem_dataset = ChromstemDataset(csv_file='output_label_test.csv',
                                     root_dir='.')

Here is a quick test to make sure the dataset looks appropriate

In [86]:
print(chromstem_dataset[20])

{'chromstem': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.8299, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0

Now I will load in the entire dataset and begin to feed it into a built neural network

In [87]:
chromstem_dataset = ChromstemDataset(csv_file='output_label.csv',
                                     root_dir='.')

To start, the dataloader will be able to shuffle the data, and I will start with a batch size of 4. No idea if that will work, but for now this is a fixed value. 

In [88]:
trainloader = DataLoader(chromstem_dataset, batch_size=1,
                        shuffle=True, num_workers=1)

testloader  = DataLoader(chromstem_dataset, batch_size=1,
                         shuffle=False, num_workers=1)

Finally, the neural network class will be initialized here.

In [97]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv3d(1,16,3)
        self.conv2 = nn.Conv3d(16,32,3)
        self.pool = nn.MaxPool3d(3,3)
        self.fc1   = nn.Linear(32*4*2,100)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()

To start, the cross entropy loss function is chosen, however a different function may be required

In [98]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

If the user has CUDA functionality on their computer, the network trains significantly faster with GPU. As such, here I will provide the opportunity to use CUDA. A correct output would say that the device being used is "cuda:0"

In [99]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

net.to(device)

cuda:0


Net(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=100, bias=True)
)

Train the network briefly here

In [100]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['chromstem'].to(device), data['num_nucls'].to(device)
    
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _,preds = torch.max(outputs,1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 3.955
[1,   200] loss: 3.036
[1,   300] loss: 2.663
[1,   400] loss: 2.501
[1,   500] loss: 2.061
[1,   600] loss: 1.810
[1,   700] loss: 1.603
[1,   800] loss: 1.617
[1,   900] loss: 1.409
[1,  1000] loss: 1.226
[1,  1100] loss: 1.357
[1,  1200] loss: 1.106
[1,  1300] loss: 0.981
[1,  1400] loss: 0.878
[1,  1500] loss: 1.026
[1,  1600] loss: 0.861
[1,  1700] loss: 0.757
[1,  1800] loss: 0.759
[1,  1900] loss: 0.723
[1,  2000] loss: 0.912
[1,  2100] loss: 0.873
[1,  2200] loss: 0.746
[1,  2300] loss: 0.726
[1,  2400] loss: 0.837
[1,  2500] loss: 0.664
[1,  2600] loss: 0.582
[1,  2700] loss: 0.882
[1,  2800] loss: 0.653
[1,  2900] loss: 0.709
[1,  3000] loss: 0.504
[1,  3100] loss: 0.768
[1,  3200] loss: 0.651
[1,  3300] loss: 0.666
[1,  3400] loss: 0.482
[1,  3500] loss: 0.401
[1,  3600] loss: 0.517
[1,  3700] loss: 0.628
[1,  3800] loss: 0.445
[1,  3900] loss: 0.544
[1,  4000] loss: 0.639
[1,  4100] loss: 0.469
[1,  4200] loss: 0.686
[1,  4300] loss: 0.456
[1,  4400] 

KeyboardInterrupt: 

With two convolutional layers of 1->16 and 16->32 channels and one output fully-connected layer, the network is able to learn the number of nucleosomes to some degree. I will now switch to python scripts instead of notebook for a more fleshed-out workflow since the preliminary work is done.